In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.0.5
Numpy : 1.18.5
Scikit-Learn : 0.23.1


Python 3.6.5 :: Anaconda, Inc.


### 전처리

In [3]:
def grap_year(data):
    data = str(data)
    return int(data[:4])
def grap_month(data):
    data = str(data)
    return int(data[4:])

In [4]:
#날짜
data = pd.read_csv('./data/201901-202003.csv')
data = data.fillna("")
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x : grap_month(x))
data =data.drop(['REG_YYMM'],axis= 1)

In [13]:
data.shape

(24697792, 13)

In [35]:
#데이터 정제
#카드이용지역_시군구, 거주지역_시군구을 제외
df = data.copy()
df = df.drop(['CARD_CCG_NM','HOM_CCG_NM'],axis= 1)

columns = ['CARD_SIDO_NM','STD_CLSS_NM', 'HOM_SIDO_NM','AGE','SEX_CTGO_CD',"FLC","year",'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [33]:
df = df.groupby(columns).sum().reset_index(drop=False)

In [28]:
df.iloc[0].to_dict()

{'CARD_SIDO_NM': '강원',
 'STD_CLSS_NM': '건강보조식품 소매업',
 'HOM_SIDO_NM': '강원',
 'AGE': '20s',
 'SEX_CTGO_CD': 1,
 'FLC': 1,
 'year': 2019,
 'month': 1,
 'CSTMR_CNT': 1,
 'AMT': 1,
 'CNT': 1}

In [36]:
df

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3
...,...,...,...,...,...,...,...,...,...,...,...
1057389,충북,휴양콘도 운영업,충북,70s,2,5,2019,3,3,148000,4
1057390,충북,휴양콘도 운영업,충북,70s,2,5,2019,5,5,329800,7
1057391,충북,휴양콘도 운영업,충북,70s,2,5,2019,10,7,557800,7
1057392,충북,휴양콘도 운영업,충북,70s,2,5,2019,12,3,247800,3


In [7]:
#인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
df_num = df.copy()
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

### EDA

### Feature Engineering & Initial Modeling

In [8]:
# Feature, taget 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT','AMT','CNT'],axis= 1)
train_target = np.log1p(train_num['AMT'])

In [12]:
df

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3
...,...,...,...,...,...,...,...,...,...,...,...
1057389,충북,휴양콘도 운영업,충북,70s,2,5,2019,3,3,148000,4
1057390,충북,휴양콘도 운영업,충북,70s,2,5,2019,5,5,329800,7
1057391,충북,휴양콘도 운영업,충북,70s,2,5,2019,10,7,557800,7
1057392,충북,휴양콘도 운영업,충북,70s,2,5,2019,12,3,247800,3


### Model Tuning & Evaluation

In [11]:
# training
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [12]:
train_features

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
149245,2,4,2,2,1,2,2019,4
554956,8,26,11,1,1,2,2020,2
918516,14,33,14,4,2,4,2019,5
425414,7,18,8,2,2,1,2019,10
640949,9,39,16,5,2,5,2020,1
...,...,...,...,...,...,...,...,...
359783,6,16,9,1,1,1,2019,12
152315,2,7,11,3,2,3,2020,3
963395,15,19,9,5,1,5,2020,2
117952,1,31,4,1,1,2,2019,7


### conclustion & Discussion

In [13]:
#예측 탬플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs = df_num['HOM_SIDO_NM'].unique()
AGEs = df_num['AGE'].unique()
SEX_CTGO_CDs = df_num['SEX_CTGO_CD'].unique()
FLCs = df_num['FLC'].unique()
years = [2020]
months = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns = train_features.columns)

In [14]:
temp.head(3)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,0,1,1,1,2020,4
1,0,0,0,1,1,1,2020,7
2,0,0,0,1,1,2,2020,4


In [15]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred,0)
temp['REG_YYMM'] = temp['year']*100 +temp['month']
temp = temp[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]
temp = temp.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).sum().reset_index(drop=False)

In [16]:
# 디코딩
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [61]:
#제출파일
submission = pd.read_csv('data/submission.csv',index_col=0)
submission = submission.drop(['AMT'],axis= 1)
submission = submission.merge(temp, left_on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'],right_on =['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'],how = 'left')
submission.index.name = 'id'


In [62]:
submission.shape

(1394, 4)

In [63]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.049486e+08
1,202004,강원,골프장 운영업,6.959584e+09
2,202004,강원,과실 및 채소 소매업,1.815465e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,9.745725e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.385428e+07
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.805813e+09
1390,202007,충북,한식 음식점업,2.552669e+10
1391,202007,충북,호텔업,2.059159e+08


### 전국업종과 비교

In [71]:
all_jobs_df = pd.read_csv('./data/외부데이터/18 사업체총조사_키워드매칭.csv',encoding = 'cp949')
all_jobs_df = all_jobs_df[['행정구역별','데이터업종명','사업체수 (개)', '종사자수 (명)']]
all_jobs_df.columns = ['CARD_SIDO_NM','STD_CLSS_NM', 'company','employee']
all_jobs_df.employee = all_jobs_df.employee.apply(lambda x : x.replace("X","0")).astype(int)
all_jobs_df = all_jobs_df.groupby(['CARD_SIDO_NM','STD_CLSS_NM'])[['company','employee']].sum().reset_index()

In [74]:
all_jobs_df

,CARD_SIDO_NM,STD_CLSS_NM,company,employee
0,강원,건강보조식품 소매업,263,399
1,강원,골프장 운영업,32,3073
2,강원,과실 및 채소 소매업,1123,2114
3,강원,관광 민예품 및 선물용품 소매업,170,261
4,강원,그외 기타 분류안된 오락관련 서비스업,208,596
...,...,...,...,...
720,충북,피자 햄버거 샌드위치 및 유사 음식점업,591,2409
721,충북,한식 음식점업,12936,33837
722,충북,호텔업,16,454
723,충북,화장품 및 방향제 소매업,718,1519


In [75]:
submission = pd.merge(submission,all_jobs_df,left_on=['CARD_SIDO_NM','STD_CLSS_NM'],right_on=['CARD_SIDO_NM','STD_CLSS_NM'],how='left')
submission.loc[submission.company.isna(),'AMT']
# submission.drop(['company','employee'],axis=1, inplace=True)

In [77]:
submission.loc[submission.company.isna()]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,company,employee
137,202004,경북,면세점,0.0,NaN,NaN
176,202004,광주,내항 여객 운송업,0.0,NaN,NaN
178,202004,광주,면세점,0.0,NaN,NaN
204,202004,광주,휴양콘도 운영업,0.0,NaN,NaN
217,202004,대구,내항 여객 운송업,0.0,NaN,NaN
245,202004,대구,휴양콘도 운영업,0.0,NaN,NaN
258,202004,대전,내항 여객 운송업,0.0,NaN,NaN
377,202004,세종,기타 수상오락 서비스업,0.0,NaN,NaN
381,202004,세종,내항 여객 운송업,0.0,NaN,NaN
383,202004,세종,면세점,0.0,NaN,NaN


In [69]:
submission.loc[submission.company.isna()]

AttributeError: 'DataFrame' object has no attribute 'company'

In [54]:
submission.shape

(1394, 4)

In [59]:
submission.index.name = 'id'
submission.to_csv('./submission/submission_사업체조사와비교.csv', encoding='utf-8-sig')

submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.049486e+08
1,202004,강원,골프장 운영업,6.959584e+09
2,202004,강원,과실 및 채소 소매업,1.815465e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,9.745725e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.385428e+07
